In [86]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [87]:
out='../output/'
inp='../input/monthly/'
pref='idm'

In [88]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
360,(2022/07/15) 23-24,0,0
361,Min,0,0
362,Max,300,78
363,Average,23.155111111111111111111111111,5.7822222222222222222222222222


In [89]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/05/04) 04-05,0.4,0.5
6,(2017/05/04) 05-06,0.0,0.0
7,(2017/05/04) 06-07,0.0,0.0
8,(2017/05/04) 07-08,0.0,0.0
9,(2017/05/04) 08-09,1.5,8.0
...,...,...,...
356,(2022/07/15) 19-20,300.0,43.0
357,(2022/07/15) 20-21,85.0,0.0
358,(2022/07/15) 21-22,0.0,0.0
359,(2022/07/15) 22-23,0.0,0.0


In [90]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/05/04) 04-05,0.4,0.5,2017-05-04,4
6,(2017/05/04) 05-06,0.0,0.0,2017-05-04,5
7,(2017/05/04) 06-07,0.0,0.0,2017-05-04,6
8,(2017/05/04) 07-08,0.0,0.0,2017-05-04,7
9,(2017/05/04) 08-09,1.5,8.0,2017-05-04,8
...,...,...,...,...,...
356,(2022/07/15) 19-20,300.0,43.0,2022-07-15,19
357,(2022/07/15) 20-21,85.0,0.0,2022-07-15,20
358,(2022/07/15) 21-22,0.0,0.0,2022-07-15,21
359,(2022/07/15) 22-23,0.0,0.0,2022-07-15,22


In [91]:
dat=dat.drop(['datehour'],1)
dat

C:\Users\kabom\AppData\Local\Temp\ipykernel_6556\4059115739.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dat=dat.drop(['datehour'],1)


,price,volume,date,hour
5,0.4,0.5,2017-05-04,4
6,0.0,0.0,2017-05-04,5
7,0.0,0.0,2017-05-04,6
8,0.0,0.0,2017-05-04,7
9,1.5,8.0,2017-05-04,8
...,...,...,...,...
356,300.0,43.0,2022-07-15,19
357,85.0,0.0,2022-07-15,20
358,0.0,0.0,2022-07-15,21
359,0.0,0.0,2022-07-15,22


In [92]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-05-04,0.4,0.5,4,2017,5,4,3
2017-05-04,0.0,0.0,5,2017,5,4,3
2017-05-04,0.0,0.0,6,2017,5,4,3
2017-05-04,0.0,0.0,7,2017,5,4,3
2017-05-04,1.5,8.0,8,2017,5,4,3
...,...,...,...,...,...,...,...
2022-07-15,300.0,43.0,19,2022,7,15,4
2022-07-15,85.0,0.0,20,2022,7,15,4
2022-07-15,0.0,0.0,21,2022,7,15,4


In [93]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume      value  price_weight_av  price_max  price_min  \
year month day                                                             
2017 5     4      41.8     57.710         1.380622       1.50        0.0   
           5     117.1    167.963         1.434355       2.00        0.0   
           6      82.8     56.174         0.678430       0.90        0.0   
           7      18.3      8.418         0.460000       0.46        0.0   
           8      69.7    104.994         1.506370       1.92        0.0   
...                ...        ...              ...        ...        ...   
2022 7     11    246.2  35700.000       145.004062     300.00        0.0   
           12    120.0  36000.000       300.000000     300.00        0.0   
           13     61.0  18300.000       300.000000     300.00        0.0   
           14    215.0  16061.000        74.702326     220.00        0.0   
           15    374.0  53358.000       142.668449     300.00        0.0   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2017 5     4      0.635000        8.0        0.0    2.090000 2017-05-04  
           5      0.884583       15.5        0.0    4.879167 2017-05-05  
           6      0.396667       11.8        0.0    3.450000 2017-05-06  
           7      0.076667        6.1        0.0    0.762500 2017-05-07  
           8      0.647500       13.0        0.0    2.904167 2017-05-08  
...                    ...        ...        ...         ...        ...  
2022 7     11    32.500000       78.0        0.0   10.258333 2022-07-11  
           12    37.500000       51.0        0.0    5.000000 2022-07-12  
           13    25.000000       39.0        0.0    2.541667 2022-07-13  
           14    20.708333       48.0        0.0    8.958333 2022-07-14  
           15    78.583333       45.0        0.0   15.583333 2022-07-15  

[1888 rows x 10 columns]

In [94]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-05-04,0.4,0.5,4,2017,5,4,3,0.2,off
2017-05-04,0.0,0.0,5,2017,5,4,3,0.0,off
2017-05-04,0.0,0.0,6,2017,5,4,3,0.0,off
2017-05-04,0.0,0.0,7,2017,5,4,3,0.0,off
2017-05-04,1.5,8.0,8,2017,5,4,3,12.0,off
...,...,...,...,...,...,...,...,...,...
2022-07-15,300.0,43.0,19,2022,7,15,4,12900.0,off
2022-07-15,85.0,0.0,20,2022,7,15,4,0.0,off
2022-07-15,0.0,0.0,21,2022,7,15,4,0.0,off


In [95]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume      value  price_weight_av  price_max  price_min  \
year month day                                                             
2017 5     4      41.8     57.710         1.380622       1.50        0.0   
           5     117.1    167.963         1.434355       2.00        0.0   
           6      82.8     56.174         0.678430       0.90        0.0   
           7      18.3      8.418         0.460000       0.46        0.0   
           8      69.7    104.994         1.506370       1.92        0.0   
...                ...        ...              ...        ...        ...   
2022 7     11    246.2  35700.000       145.004062     300.00        0.0   
           12    120.0  36000.000       300.000000     300.00        0.0   
           13     61.0  18300.000       300.000000     300.00        0.0   
           14    215.0  16061.000        74.702326     220.00        0.0   
           15    374.0  53358.000       142.668449     300.00        0.0   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2017 5     4      0.635000        8.0        0.0    2.090000 2017-05-04  ...   
           5      0.884583       15.5        0.0    4.879167 2017-05-05  ...   
           6      0.396667       11.8        0.0    3.450000 2017-05-06  ...   
           7      0.076667        6.1        0.0    0.762500 2017-05-07  ...   
           8      0.647500       13.0        0.0    2.904167 2017-05-08  ...   
...                    ...        ...        ...         ...        ...  ...   
2022 7     11    32.500000       78.0        0.0   10.258333 2022-07-11  ...   
           12    37.500000       51.0        0.0    5.000000 2022-07-12  ...   
           13    25.000000       39.0        0.0    2.541667 2022-07-13  ...   
           14    20.708333       48.0        0.0    8.958333 2022-07-14  ...   
           15    78.583333       45.0        0.0   15.583333 2022-07-15  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2017 5     4                   NaN                 NaN        41.8   
           5                   NaN                 NaN       117.1   
           6                   NaN                 NaN        82.8   
           7                   NaN                 NaN        18.3   
           8                   NaN                 NaN        69.7   
...                            ...                 ...         ...   
2022 7     11                  NaN                 NaN       246.2   
           12                  NaN                 NaN       120.0   
           13                  NaN                 NaN        61.0   
           14                  NaN                 NaN       215.0   
           15                  NaN                 NaN       374.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2017 5     4                NaN             NaN             NaN     57.710   
           5                NaN             NaN             NaN    167.963   
           6                NaN             NaN             NaN     56.174   
           7                NaN             NaN             NaN      8.418   
           8                NaN             NaN             NaN    104.994   
...                         ...             ...             ...        ...   
2022 7     11               NaN             NaN             NaN  35700.000   
           12               NaN             NaN             NaN  36000.000   
           13               NaN             NaN             NaN  18300.000   
           14               NaN             NaN             NaN  16061.000   
           15               NaN             NaN             NaN  53358.000   

                standard_value  morning_value  evening_value

In [96]:
daily.to_csv(out + pref + '_daily.csv')

In [97]:
hourly.to_csv(out + pref + '_hourly.csv')

In [98]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-05-04,4,0.4,0.5,off
2017-05-04,5,0.0,0.0,off
2017-05-04,6,0.0,0.0,off
2017-05-04,7,0.0,0.0,off
2017-05-04,8,1.5,8.0,off
...,...,...,...,...
2022-07-15,19,300.0,43.0,off
2022-07-15,20,85.0,0.0,off
2022-07-15,21,0.0,0.0,off


In [99]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [100]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_6556\3018208690.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)


,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,5,133.892857,169.184429,1.263000,0.0,0.765813,15.625000,0.235714,5.705804,...,0.0,1.263581,1.263581,NaN,NaN,NaN,5.714939,NaN,NaN,NaN
1,2017,6,69.630000,124.327867,1.224500,0.0,0.670972,15.276667,0.000000,3.046806,...,0.0,1.785550,1.785550,NaN,NaN,NaN,3.001293,NaN,NaN,NaN
2,2017,7,65.567742,84.076677,1.192083,0.0,0.543903,11.129032,0.016129,2.842742,...,0.0,1.282287,1.282287,NaN,NaN,NaN,2.838827,NaN,NaN,NaN
3,2017,8,51.993548,67.690129,1.337500,0.0,0.485046,10.132258,0.170968,2.314220,...,0.0,1.301895,1.301895,NaN,NaN,NaN,2.251117,NaN,NaN,NaN
4,2017,9,73.550000,79.726867,0.987083,0.0,0.432450,11.293333,0.000000,3.172472,...,0.0,1.083982,1.083982,NaN,NaN,NaN,3.170259,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2022,3,377.483871,17052.145161,45.170000,0.0,13.143118,41.000000,2.612903,17.498925,...,0.0,45.173176,45.173176,NaN,NaN,NaN,16.343575,NaN,NaN,NaN
59,2022,4,295.517241,16206.275862,46.416500,0.0,10.275124,49.137931,2.241379,12.923851,...,0.0,54.840373,54.840373,NaN,NaN,NaN,12.677515,NaN,NaN,NaN
60,2022,5,278.580000,16302.191200,51.451250,0.0,12.128117,47.960000,0.000000,11.615500,...,0.0,58.518886,58.518886,NaN,NaN,NaN,11.764358,NaN,NaN,NaN
61,2022,6,251.074074,23227.271111,75.926667,0.0,19.903765,53.925926,0.000000,10.669753,...,0.0,92.511627,92.511627,NaN,NaN,NaN,10.658805,NaN,NaN,NaN


In [101]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,5,133.892857,169.184429,1.263000,0.0,0.765813,15.625000,0.235714,5.705804,...,0.0,1.263581,1.263581,NaN,NaN,NaN,5.714939,NaN,NaN,NaN
1,2017,6,69.630000,124.327867,1.224500,0.0,0.670972,15.276667,0.000000,3.046806,...,0.0,1.785550,1.785550,NaN,NaN,NaN,3.001293,NaN,NaN,NaN
2,2017,7,65.567742,84.076677,1.192083,0.0,0.543903,11.129032,0.016129,2.842742,...,0.0,1.282287,1.282287,NaN,NaN,NaN,2.838827,NaN,NaN,NaN
3,2017,8,51.993548,67.690129,1.337500,0.0,0.485046,10.132258,0.170968,2.314220,...,0.0,1.301895,1.301895,NaN,NaN,NaN,2.251117,NaN,NaN,NaN
4,2017,9,73.550000,79.726867,0.987083,0.0,0.432450,11.293333,0.000000,3.172472,...,0.0,1.083982,1.083982,NaN,NaN,NaN,3.170259,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2022,3,377.483871,17052.145161,45.170000,0.0,13.143118,41.000000,2.612903,17.498925,...,0.0,45.173176,45.173176,NaN,NaN,NaN,16.343575,NaN,NaN,NaN
59,2022,4,295.517241,16206.275862,46.416500,0.0,10.275124,49.137931,2.241379,12.923851,...,0.0,54.840373,54.840373,NaN,NaN,NaN,12.677515,NaN,NaN,NaN
60,2022,5,278.580000,16302.191200,51.451250,0.0,12.128117,47.960000,0.000000,11.615500,...,0.0,58.518886,58.518886,NaN,NaN,NaN,11.764358,NaN,NaN,NaN
61,2022,6,251.074074,23227.271111,75.926667,0.0,19.903765,53.925926,0.000000,10.669753,...,0.0,92.511627,92.511627,NaN,NaN,NaN,10.658805,NaN,NaN,NaN


In [102]:
monthly=monthly.set_index(['year','month'])

In [103]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [104]:
monthly.to_csv(out + pref + '_monthly.csv')

In [105]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [106]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_6556\2677402245.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)


,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,off_price_mean,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,25380.363787,37.796634,42.035742,0.250581,16.671295,65.063871,0.216774,21.664290,16.671295,...,0.0,786791.277400,716.000000,0.0,0.0,0.0,21.707207,NaN,NaN,NaN
1,2,12506.608214,35.458551,34.499214,0.171429,10.147189,39.125714,0.142857,9.927369,10.147189,...,0.0,350185.030000,656.000000,0.0,0.0,0.0,9.814946,NaN,NaN,NaN
2,3,14070.070568,34.008527,34.560129,0.462323,11.631758,39.643871,0.792903,12.304452,11.631758,...,0.0,436172.187600,716.000000,0.0,0.0,0.0,11.878101,NaN,NaN,NaN
3,4,18845.699094,43.610266,51.300067,0.698456,15.432601,56.203356,1.165772,15.994351,15.432601,...,0.0,561601.833000,692.000000,0.0,0.0,0.0,15.806825,NaN,NaN,NaN
4,5,11892.317718,37.403377,46.099718,0.279661,9.815703,47.005650,0.758192,10.775278,9.815703,...,0.0,350823.372667,685.333333,0.0,0.0,0.0,10.744292,NaN,NaN,NaN
5,6,27684.368141,55.266761,84.127006,1.665480,22.726500,57.402260,0.875706,14.204204,22.726500,...,0.0,816688.860167,686.000000,0.0,0.0,0.0,14.033651,NaN,NaN,NaN
6,7,27066.665182,49.562316,76.094941,0.014471,20.186739,70.804706,0.015882,18.296613,20.186739,...,0.0,766888.846833,656.000000,0.0,0.0,0.0,17.046075,NaN,NaN,NaN
7,8,22105.735548,51.566474,69.030258,0.542323,16.897351,49.944516,0.582581,14.465978,16.897351,...,0.0,685277.802000,716.000000,0.0,0.0,0.0,14.559106,NaN,NaN,NaN
8,9,17160.815747,30.871534,36.370333,0.456733,12.027207,49.975333,0.242000,16.184989,12.027207,...,0.0,514824.472400,696.000000,0.0,0.0,0.0,16.172931,NaN,NaN,NaN
9,10,25822.894768,38.273415,51.913806,1.211548,16.323140,68.749032,0.751613,22.436333,16.323140,...,0.0,800509.737800,716.000000,0.0,0.0,0.0,22.545978,NaN,NaN,NaN


In [107]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [108]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_6556\1619064361.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)


price     volume        value
month hour                                   
1     0      5.986750   7.544167   228.427008
      1      7.118833   8.294167   266.413675
      2      7.651000  10.869167   351.861675
      3      7.900250  12.140000   396.614258
      4      8.334903  14.364516   457.878148
...               ...        ...          ...
12    19    27.068000  11.377419   974.025697
      20    20.872581  13.998710  1216.757116
      21    12.206774  12.587742   643.392839
      22     4.075548   5.479355   146.764123
      23     6.085419  10.083226   302.208942

[288 rows x 3 columns]

In [109]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [110]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_6556\4270116260.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)


price     volume        value
year month hour                                    
2017 5     0       0.254167   1.041667     0.570667
           1       0.221667   0.933333     0.499500
           2       0.194167   0.866667     0.456500
           3       0.268333   1.170833     0.621167
           4       0.343214   2.917857     1.696500
...                     ...        ...          ...
2022 7     19    156.000000  18.220000  5358.000000
           20     25.666667   3.400000  1020.000000
           21      5.666667   4.866667   219.666667
           22      0.000000   0.000000     0.000000
           23      3.266667   2.600000    63.066667

[1512 rows x 3 columns]

In [111]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [112]:
from statsmodels.tsa.stattools import pacf

In [113]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.0,1.0,1.0,1.000000
1,NaN,NaN,NaN,0.720872
2,NaN,NaN,NaN,0.369981
3,NaN,NaN,NaN,0.017553
4,NaN,NaN,NaN,0.045506
5,NaN,NaN,NaN,0.174380
6,NaN,NaN,NaN,0.004575
7,NaN,NaN,NaN,0.007844
8,NaN,NaN,NaN,-0.073251
9,NaN,NaN,NaN,0.059888


In [114]:
pacfs.to_csv(out + pref + '_pacfs.csv')